## Building binary decision trees from scratch and analyzing effects of regularizations


In this project, I use past data from peer-to-peer lending company the LeandingClub, to predict whether a future loan is likely to be paid off or default. 

I use this dataset to practice implementing decesion tree from scratch and to play with various regularization methods.

- implement decision tree with numpy and pandas
- implement various methods of regularization to prevent overfitting
- use the loan defaults data to analysis the effect of limiting (1) maximum tree depth (2) minimum node size (3) minimum error gain

The data is provided as part of the UW classification class. I extended the class project into a more thorough analysis presented in this notebook.

Emma Yu

In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8., 10.0) 

In [2]:
loans = pd.read_csv('lending-club-data.csv') 
print loans.shape
loans.head()

(122607, 68)


/Users/emmayu/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1,1,1,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1,1,1,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1,1,1,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1,1,1,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1,1,1,0,5.21533,20141201T000000,1,1,1


In [3]:
num_bad = (loans['bad_loans'] == 1).sum()
num_total = loans['bad_loans'].count()
print 'number of bad loans = ', num_bad
print 'bad loan rate = ', float(num_bad)/num_total
print 'good loan rate = ', 1-float(num_bad)/num_total

number of bad loans =  23150
bad loan rate =  0.188814668004
good loan rate =  0.811185331996


In [4]:
# For the purpose of this exercise, we only consider 4 features

features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'bad_loans'

In [5]:
# Extract the feature columns and target column
loans = loans[features + [target]]
loans.head()

,grade,term,home_ownership,emp_length,bad_loans
0,B,36 months,RENT,10+ years,0
1,C,60 months,RENT,< 1 year,1
2,C,36 months,RENT,10+ years,0
3,C,36 months,RENT,10+ years,0
4,A,36 months,RENT,3 years,0


### Preprocessing

The goal to make a binary decision tree, so we need preprocess categorical variables and make dummy variables for each category.

In [6]:
# Let's try this again with get dummies 
loans = pd.get_dummies(loans)

In [7]:
len(loans.columns.values)

26

In [11]:
#Now let's separate the training and validation data. 
import json
with open('module-5-assignment-2-train-idx.json') as train_file:    
    train_index = json.load(train_file)
with open('module-5-assignment-2-test-idx.json') as valid_file:    
    valid_index = json.load(valid_file)

In [12]:
print len(train_index)
print len(valid_index)

37224
9284


In [13]:
train_data= loans.iloc[train_index]
valid_data= loans.iloc[valid_index]

In [14]:
cols = [col for col in train_data.columns if col not in ['bad_loans']]
train_x = train_data[cols]
train_y = train_data['bad_loans']

cols = [col for col in valid_data.columns if col not in ['bad_loans']]
valid_x = valid_data[cols]
valid_y = valid_data['bad_loans']

In [20]:
cols

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year',
 'emp_length_n/a']

### Building a decision tree

In [12]:
def num_mistakes(labels):
    '''
    for one node, count the number of mistakes for a majority classifer
    '''
    # corner case, empty variable passed
    if len(labels) == 0:
        return 0
    
    num_bad = (labels == 1).sum()
    num_good = (labels == 0).sum()
    return min(num_bad, num_good)

In [13]:
def pick_feature(data, features, labels):
    '''
    For one node in the tree, loop through features and pick the one with 
    smallest splitting error to split on 
    
    Split based on error function now, let's try to impliment Gini index and entropy later
    '''
    min_error = len(labels)
    fea_split = ''
    for f in features:
        #print f
        group1 = data[data[f] == 1]
        group1_labels = labels[data[f] == 1]
        
        group2 = data[data[f] == 0]
        group2_labels = labels[data[f] == 0]
        
        error = num_mistakes(group1_labels) + num_mistakes(group2_labels)
        
        if error < min_error:
            min_error = error
            fea_split = f          
    
    return fea_split

In [14]:
#pick_feature(train_x, cols, train_y)

In [15]:
def create_leaf(labels):    
    '''
    create a leaf node for splitting recursively 
    for labels, 1 means bad loan and 0 means good loan.
    '''
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf':  True}   
   
    # Count the number of data points that are 1 and 0 in this node.
    num_pos = len(labels[labels == 1])
    num_neg = len(labels[labels == 0])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_pos > num_neg:
        leaf['prediction'] =  1       
    else:
        leaf['prediction'] =  0            
    return leaf 

At each split we want to drop one feature
At each split we double the number of leave nodes (does it mean we need to create a new round of leave nodes?)

In [13]:
def decision_tree_create(data, features, labels, current_depth = 0, max_depth = 10):
    ''' Build the tree in a depth first way'''

    remaining_features = features[:] 

    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(labels))
    
    # Stopping condition 1 - if the node is pure
    if  num_mistakes(labels)== 0: 
        print "Stop splitting because the node is pure"     
        return create_leaf(labels)
    
    # Stopping condition 2 - running out of features
    if remaining_features == []:   
        print "Stop splitting because no feature is available"    
        return create_leaf(labels)  

    # Stopping condition 3 - if maximum depth has reached
    if current_depth >= max_depth:  
        print "Stop splitting because the tree has reached the maximum depth."
        return create_leaf(labels)

    # Split the tree on the feature that minimizes the error
    # note that the features and ys are stored separately 
    # we want to make sure to split on both
    splitting_feature = pick_feature(data, features, labels)
    left_split = data[data[splitting_feature] == 0]
    left_labels = labels[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1] 
    right_labels = labels[data[splitting_feature] == 1] 
    
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)"%(splitting_feature, len(left_split), len(right_split))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_labels)
    if len(right_split) == len(data):
        print "Creating leaf node."
        return create_leaf(right_labels)

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, left_labels, current_depth + 1, max_depth)        
    right_tree = decision_tree_create(right_split, remaining_features, right_labels, current_depth + 1, max_depth) 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [98]:
tree = decision_tree_create(train_x, cols, train_y, current_depth = 0, max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
S

In [99]:
tree

{'is_leaf': False,
 'left': {'is_leaf': False,
  'left': {'is_leaf': False,
   'left': {'is_leaf': False,
    'left': {'is_leaf': False,
     'left': {'is_leaf': False,
      'left': {'is_leaf': True,
       'left': None,
       'prediction': 1,
       'right': None,
       'splitting_feature': None},
      'prediction': None,
      'right': {'is_leaf': True,
       'left': None,
       'prediction': 1,
       'right': None,
       'splitting_feature': None},
      'splitting_feature': 'grade_E'},
     'prediction': None,
     'right': {'is_leaf': True,
      'left': None,
      'prediction': 1,
      'right': None,
      'splitting_feature': None},
     'splitting_feature': 'grade_D'},
    'prediction': None,
    'right': {'is_leaf': True,
     'left': None,
     'prediction': 1,
     'right': None,
     'splitting_feature': None},
    'splitting_feature': 'grade_C'},
   'prediction': None,
   'right': {'is_leaf': False,
    'left': {'is_leaf': True,
     'left': None,
     'predictio

In [18]:
def classify(tree, x, annotate = False):
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [107]:
print valid_x.iloc[0]
print 'Predicted class: %s ' % classify(tree,valid_x.iloc[0], annotate = True)


grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
emp_length_n/a             0
Name: 24, dtype: float64
Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
At leaf, predicting 1
Predicted class: 1 


In [101]:
classify(tree,valid_x.iloc[0])

1

In [23]:
def evaluate(tree, data, ys):
    ''' 
    Apply the classify(tree, x) to each row in the data
    then calculate the error rate
    '''
    length = len(ys)
    prediction = np.zeros(length)
    correct = 0 
    
    for i in range(length):
        prediction[i] = classify(tree, data.iloc[i])
        if prediction[i] == ys.iloc[i]:
            correct += 1
            
    error = (length-correct)/float(length)
    return prediction, error
    

In [103]:
print evaluate(tree, valid_x, valid_y)
#len(valid_y)

(array([ 1.,  0.,  1., ...,  1.,  1.,  0.]), 0.3837785437311504)


In [115]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print "(leaf, label: %s)" % tree['prediction']
        return None
    print split_name
    split_feature, split_value = split_name.split('.')
    print '                       %s' % name
    print '         |---------------|----------------|'
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '  [{0} == 0]               [{0} == 1]    '.format(split_name)
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree'))

In [119]:
tree.viewkeys

<function viewkeys>

In [110]:
#print_stump(tree['left'], tree['splitting_feature'])

In [121]:
#print_stump(tree['left']['left'], tree['left']['splitting_feature'])

### Early stop to avoid over-fitting

In order to prevent over-fitting, we want to implement 3 early stop conditions:
- the tree has reached a maximum depth (inplemented already)
- number of elements contained has reached a minimum (minimum node size)
- do not gain enough error reduction by splitting 

In [30]:
def decision_tree_create_es(data, features, labels, current_depth = 0, max_depth = 10, min_node_size = 10, min_error_gain = 0.1):
    ''' 
    Build the tree in a depth first way
    es - adding two early stop conditions 
    '''

    remaining_features = features[:] 

    #print "--------------------------------------------------------------------"
    #print "Subtree, depth = %s (%s data points)." % (current_depth, len(labels))
    
    # Stopping condition 1 - if the node is pure
    if  num_mistakes(labels)== 0: 
        #print "Stop splitting because the node is pure"     
        return create_leaf(labels)
    
    # Stopping condition 2 - running out of features
    if remaining_features == []:   
        #print "Stop splitting because no feature is available"    
        return create_leaf(labels)  

    # Stopping condition 3 - if maximum depth has reached
    if current_depth >= max_depth:  
        #print "Stop splitting because the tree has reached the maximum depth."
        return create_leaf(labels)
    
    # Early stop condition 2 - if the minimum node size has been reached
    if len(labels) <= min_node_size:
        #print "Stop splitting because the tree has reached the minimum node size."
        return create_leaf(labels)
    

    # Split the tree on the feature that minimizes the error
    # note that the features and ys are stored separately 
    # we want to make sure to split on both
    splitting_feature = pick_feature(data, features, labels)
    left_split = data[data[splitting_feature] == 0]
    left_labels = labels[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1] 
    right_labels = labels[data[splitting_feature] == 1] 
    
    # Early stop condition 3 - if the gain in error reduction is big enough
    error_before = num_mistakes(labels)
    error_after = num_mistakes(left_labels) + num_mistakes(right_labels)
    if float(error_before - error_after)/len(labels) <= min_error_gain:
        #print 'Stop splitting because not having big enough error gain'
        return create_leaf(labels)

    
    remaining_features.remove(splitting_feature)
    #print "Split on feature %s. (%s, %s)"%(splitting_feature, len(left_split), len(right_split))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        #print "Creating leaf node."
        return create_leaf(left_labels)
    if len(right_split) == len(data):
        #print "Creating leaf node."
        return create_leaf(right_labels)

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create_es(left_split, remaining_features, left_labels, current_depth + 1, max_depth, min_node_size, min_error_gain)        
    right_tree = decision_tree_create_es(right_split, remaining_features, right_labels, current_depth + 1, max_depth, min_node_size, min_error_gain) 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [8]:
# for the model with early stop conditions, we want to use a different sub-sample just for fun.
import json
with open('module-6-assignment-train-idx.json') as train_file_es:    
    train_index_es = json.load(train_file_es)
with open('module-6-assignment-validation-idx.json') as valid_file_es:    
    valid_index_es = json.load(valid_file_es)
    
print len(train_index_es)
print len(valid_index_es)

train_data_es= loans.iloc[train_index_es]
valid_data_es= loans.iloc[valid_index_es]

cols = [col for col in train_data_es.columns if col not in ['bad_loans']]
train_x_es = train_data_es[cols]
train_y_es = train_data_es['bad_loans']

cols = [col for col in valid_data_es.columns if col not in ['bad_loans']]
valid_x_es = valid_data_es[cols]
valid_y_es = valid_data_es['bad_loans']

37224
9284


In [10]:
train_x_es.shape

(37224, 25)

Now let's train a model with minimum node size of 100. 

In [16]:
tree_es = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 6, min_node_size = 100, min_error_gain=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Stop splitting because not having big enough error gain
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_n/a. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Stop splitting because the tree has reached the minimum node size.
--------------------------------------------------------------------
Subtree, depth = 3 (5 data points).
Stop splitting because the tree has reached the minimum node size.
----------------------------------------

In [20]:
print valid_x_es.iloc[0]
print 'Predicted class: %s ' % classify(tree_es,valid_x_es.iloc[0], annotate = True)

grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
emp_length_n/a             0
Name: 24, dtype: float64
Split on term_ 36 months = 0.0
Split on grade_A = 0.0
At leaf, predicting 1
Predicted class: 1 


In [132]:
len(cols)

25

In [21]:
# build a tree without early stop condition 2
tree_comparison = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 6, min_node_size = 0, min_error_gain=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
S

In [22]:
print valid_x_es.iloc[0]
print 'Predicted class: %s ' % classify(tree_comparison,valid_x_es.iloc[0], annotate = True)

grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
emp_length_n/a             0
Name: 24, dtype: float64
Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
At leaf, predicting 1
Predicted class: 1 


Now let's compare the results between the previous tree we built without early minimum node size and the one with minimum node size of 100.

In [38]:
#the new tree 
print 'min node size 100, train: ',evaluate(tree_es, train_x_es, train_y_es)
print 'min node size 100, valid: ',evaluate(tree_es, valid_x_es, valid_y_es)
# the old tree
print 'min node size 0, train: ', evaluate(tree_comparison, train_x_es, train_y_es)
print 'min node size 0, valid: ', evaluate(tree_comparison, valid_x_es, valid_y_es)

min node size 100, train:  (array([ 1.,  1.,  1., ...,  0.,  1.,  1.]), 0.38203846980442724)
min node size 100, valid:  (array([ 1.,  0.,  1., ...,  1.,  1.,  0.]), 0.38367083153813014)
min node size 0, train:  (array([ 1.,  1.,  1., ...,  0.,  1.,  1.]), 0.38185041908446166)
min node size 0, valid:  (array([ 1.,  0.,  1., ...,  1.,  1.,  0.]), 0.3837785437311504)


We can see the old tree with minimum node size 0 has smaller training error but higher validation error. This could be a sign of overfitting.

### To further study the effect of each parameter, let's build three groups of trees. 
In the first group, let's isolate the effect of maximum tree depth, and look at the error rate of trees with maxinum depth of 2, 6, and 14.

In [31]:
# Parameter stud, lets train a suite of trees
tree_depth2 = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 2, min_node_size = 0, min_error_gain=-1)
tree_depth6 = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 6, min_node_size = 0, min_error_gain=-1)
tree_depth14 = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 14, min_node_size = 0, min_error_gain=-1)

In [33]:
# 2 level tree
print 'level2, train: ',evaluate(tree_depth2, train_x_es, train_y_es)
print 'level2, valid: ',evaluate(tree_depth2, valid_x_es, valid_y_es)

# 6 level tree
print 'level6, train: ',evaluate(tree_depth6, train_x_es, train_y_es)
print 'level6, valid: ',evaluate(tree_depth6, valid_x_es, valid_y_es)

# 14 level tree
print 'level14, train: ',evaluate(tree_depth14, train_x_es, train_y_es)
print 'level14, valid: ',evaluate(tree_depth14, valid_x_es, valid_y_es)


level2, train:  (array([ 1.,  1.,  1., ...,  0.,  1.,  1.]), 0.40003761014399314)
level2, valid:  (array([ 1.,  0.,  1., ...,  1.,  0.,  0.]), 0.3981042654028436)
level6, train:  (array([ 1.,  1.,  1., ...,  0.,  1.,  1.]), 0.38185041908446166)
level6, valid:  (array([ 1.,  0.,  1., ...,  1.,  1.,  0.]), 0.3837785437311504)
level14, train:  (array([ 1.,  1.,  1., ...,  0.,  1.,  1.]), 0.3761820330969267)
level14, valid:  (array([ 1.,  0.,  1., ...,  1.,  1.,  0.]), 0.37731581214993537)


We can see both the training error and the validation error decrease with tree depth in this example. This means even with a tree of depth 14, we are still making progress by building a larger tree, and overfitting does not seem to be an issue here.



#### Now let's look at error reduction.
We consider trees with minimum error reduction of -1, 0 and 5. This means we will not split a node unless the maximum gain on error is larger than this minimum.

In [34]:
tree_error1 = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 6, min_node_size = 0, min_error_gain=-1)
tree_error2 = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 6, min_node_size = 0, min_error_gain=0)
tree_error3 = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 6, min_node_size = 0, min_error_gain=5)

In [35]:
print 'error reduction = -1, train: ',evaluate(tree_error1, train_x_es, train_y_es)
print 'error reduction = -1, valid: ',evaluate(tree_error1, valid_x_es, valid_y_es)

print 'error reduction = 0, train: ',evaluate(tree_error2, train_x_es, train_y_es)
print 'error reduction = 0, valid: ',evaluate(tree_error2, valid_x_es, valid_y_es)

print 'error reduction = 5, train: ',evaluate(tree_error3, train_x_es, train_y_es)
print 'error reduction = 5, valid: ',evaluate(tree_error3, valid_x_es, valid_y_es)


error reduction = -1, train:  (array([ 1.,  1.,  1., ...,  0.,  1.,  1.]), 0.38185041908446166)
error reduction = -1, valid:  (array([ 1.,  0.,  1., ...,  1.,  1.,  0.]), 0.3837785437311504)
error reduction = 0, train:  (array([ 1.,  1.,  1., ...,  0.,  1.,  1.]), 0.3819578766387277)
error reduction = 0, valid:  (array([ 1.,  0.,  1., ...,  1.,  1.,  0.]), 0.3837785437311504)
error reduction = 5, train:  (array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), 0.4963464431549538)
error reduction = 5, valid:  (array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), 0.503446790176648)


In [36]:
tree_node1 = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 6, min_node_size = 0, min_error_gain=-1)
tree_node2 = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 6, min_node_size = 2000, min_error_gain=-1)
tree_node3 = decision_tree_create_es(train_x_es, cols, train_y_es, max_depth = 6, min_node_size = 50000, min_error_gain=-1)

In [37]:
print 'min node size = 0, train: ',evaluate(tree_node1, train_x_es, train_y_es)
print 'min node size = 0, valid: ',evaluate(tree_node1, valid_x_es, valid_y_es)

print 'min node size = 2000, train: ',evaluate(tree_node2, train_x_es, train_y_es)
print 'min node size = 2000, valid: ',evaluate(tree_node2, valid_x_es, valid_y_es)

print 'min node size = 50000, train: ',evaluate(tree_node3, train_x_es, train_y_es)
print 'min node size = 50000, valid: ',evaluate(tree_node3, valid_x_es, valid_y_es)

min node size = 0, train:  (array([ 1.,  1.,  1., ...,  0.,  1.,  1.]), 0.38185041908446166)
min node size = 0, valid:  (array([ 1.,  0.,  1., ...,  1.,  1.,  0.]), 0.3837785437311504)
min node size = 2000, train:  (array([ 1.,  1.,  1., ...,  0.,  1.,  1.]), 0.3837040618955513)
min node size = 2000, valid:  (array([ 1.,  0.,  1., ...,  1.,  1.,  1.]), 0.38453252908229213)
min node size = 50000, train:  (array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), 0.4963464431549538)
min node size = 50000, valid:  (array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), 0.503446790176648)
